In [ ]:
#This code is written by myself with the help of online resources

In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

In [ ]:
data_path = r'C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\50_input_set'

In [ ]:
test_dir = r'C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\Car_test_from_mobilenet_SSD\Test_dir'

In [ ]:
train_dir = os.path.join(data_path,'train')
val_dir = os.path.join(data_path,'validation')

In [ ]:
train_dir, val_dir,test_dir

In [ ]:
IMG_SIZE =(160,160)
BATCH_SIZE =5

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE
)

In [ ]:
  class_names = train_dataset.class_names

  plt.figure(figsize=(10,10))
  for images, labels in train_dataset.take(1):
    for i in range(5):
      ax = plt.subplot(3,3,i + 1)
      plt.imshow(images[i].numpy().astype("uint8"))
      plt.title(class_names[labels[i]])
      plt.axis("off")

In [ ]:

 validation_dataset = val_dataset

In [ ]:
print('Number of validation batches:%d' %tf.data.experimental.cardinality(validation_dataset))


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset =validation_dataset.prefetch(buffer_size=AUTOTUNE)


In [ ]:
data_aug = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2)
])

In [ ]:
for image, labels in train_dataset.take(1):
  plt.figure(figsize=(10,10))
  first_image = image[3]
  for i in range(9):
    ax =plt.subplot(3,3,i+1)
    augmented_image = data_aug(tf.expand_dims(first_image,0))
    plt.imshow(augmented_image[0]/255)
    plt.axis('off')

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape =IMG_SHAPE,
    include_top = False,
    weights = 'imagenet',
)

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature=base_model(image_batch)
print(feature.shape)

In [ ]:
base_model.trainable = False

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(160,160,3))
x = data_aug(inputs)
x= preprocess_input(x)
x=base_model(x,training = False)
x=global_average_layer(x)
x=tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs,outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics = ['accuracy']


)

In [ ]:
model.summary()

In [ ]:
initial_epochs=20
#loss0,acc0 = model.evaluate(test_dataset)

In [ ]:
#print('initial loss:{:.2f}'.format(loss0))
#print('initial acc:{:.2f}'.format(acc0))

In [ ]:
history = model.fit(
    train_dataset,
    epochs = initial_epochs,
    validation_data = validation_dataset
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:

print("Number of layers in the base model: ", len(base_model.layers))


fine_tune_at = 100


for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
    metrics=['accuracy']
)

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=validation_dataset
)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
#loss, accuracy = model.evaluate(test_dataset)
#print('Test accuracy :', accuracy)

In [ ]:
def fun():
    test_dir = r'C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\Car_test_from_mobilenet_SSD\test1'
    IMG_SIZE =(160,160)
    global test_dataset
    test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=False,
    image_size = IMG_SIZE)
    test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)
    

    

In [ ]:
def pred():
    image_batch,label_batch= test_dataset.as_numpy_iterator().next()
    predictions = model.predict_on_batch(image_batch).flatten()
    #print(predictions)
    predictions = tf.nn.sigmoid(predictions)
    #print(predictions)
    predictions = tf.where(predictions < 0.5, 0, 1)
    #print('Predictions:\n', predictions.numpy())
    return predictions
    

In [ ]:
import numpy as np
def non_max_suppression_fast(boxes, overlapThresh):
    if len(boxes) == 0:
        return []
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    pick = []
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last],np.where(overlap > overlapThresh)[0])))
    return boxes[pick].astype("int")
                       



In [ ]:
#This is incorprating the sliding window approach to perform identification of object

In [ ]:
from PIL import Image
import os
import cv2
import uuid
import time
input = r"C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\Car_test_from_mobilenet_SSD\Test_dir\Test_images"
start_time= time.time()
for file in os.listdir(input):
    
    

    path=r"C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\Car_test_from_mobilenet_SSD\test"
    new_path =r"C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\Car_test_from_mobilenet_SSD\test1\input"
    height =187
    width =310
    k = 1
    im = Image.open(os.path.join(input,file))
    img =[]
    g=0
    

    z=[]
    r =[]
    imgwidth, imgheight = im.size
    i=93
    for j in range(155,imgwidth,width):
            if imgwidth-j<200 or imgheight-i<10:
                break
            else:
                #print(j)
                h = j+width
                m = i+height
                if (imgheight - m < 5):
                    box =(j,i,h,imgheight)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)

                elif (imgwidth - h <5):
                    box =(j,i,imgwidth,m)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                else:
                    box = (j, i, h, m)
                    a = im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                try:
                    o = a.crop(area)
                    #print(o)
                    o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                except:
                    pass
                k +=1

    for j in range(0,imgwidth,width):
            if imgwidth-j<10 or imgheight-i<10:
                break
            else:
                #print(j)
                h = j+width
                m = i+height
                if (imgheight - m < 5):
                    box =(j,i,h,imgheight)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)

                elif (imgwidth - h <5):
                    box =(j,i,imgwidth,m)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                else:
                    box = (j, i, h, m)
                    a = im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                try:
                    o = a.crop(area)
                    #print(o)
                    o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
                except:
                    pass
                k +=1
    for i in range(0,imgheight,height):
        for j in range(0,imgwidth,width):
            if imgwidth-j<10 or imgheight-i<10:
                break
            else:
                #print(j)
                h = j+width
                m = i+height
                if (imgheight - m < 5):
                    box =(j,i,h,imgheight)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)

                elif (imgwidth - h <5):
                    box =(j,i,imgwidth,m)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                else:
                    box = (j, i, h, m)
                    a = im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                try:
                    o = a.crop(area)
                    #print(o)
                    o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                except:
                    pass
                k +=1
    for i in range(0,imgheight,height):
        for j in range(155,imgwidth,width):
            if imgwidth-j<200 or imgheight-i<10:
                break
            else:
                #print(j)
                h = j+width
                m = i+height
                if (imgheight - m < 5):
                    box =(j,i,h,imgheight)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)

                elif (imgwidth - h <5):
                    box =(j,i,imgwidth,m)
                    a =im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                else:
                    box = (j, i, h, m)
                    a = im.crop(box)
                    z.append(box)
                    img.append(a)
                    a.save(os.path.join(path,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                    fun()
                    f = pred()
                    if f==g:
                        r.append(box)
                try:
                    o = a.crop(area)
                    #print(o)
                    o.save(os.path.join(path,"PNG","%s" % page,"IMG-%s.png" % k))
                    a.save(os.path.join(new_path,"IMG.png"))
                except:
                    pass
                k +=1
    #print(z)
    r= np.array(r)
    q= non_max_suppression_fast(r,0.49)
    v=1
    Res_Path = r"C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\Transfer_Learning\Car_test_from_mobilenet_SSD\Result_save"
    
    imag=cv2.imread(os.path.join(input,file))
    for i in range (len(q)):
        left = q[i][:2]
        right =q[i][2:4]
    cv2.rectangle(imag,left,right,(0,255,0),thickness=2)
    cv2.imwrite(os.path.join(Res_Path,'{}.png'.format(uuid.uuid1())),imag)
    v +=1
stop_time=time.time()
duration =stop_time - start_time
hours = duration // 3600
minutes = (duration - (hours * 3600)) // 60
seconds = duration - ((hours * 3600) + (minutes * 60))
msg = f'(training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
print (msg, flush=True) # p


In [ ]:
print(r)
print(type(r))
r= np.array(r)
print(type(r))

In [ ]:
q= non_max_suppression_fast(r,0.49)

In [ ]:
print(q)

In [ ]:
import cv2
img =cv2.imread(r"C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\test_image\000026.png")

for i in range (len(q)):
    left = q[i][:2]
    right =q[i][2:4]
    cv2.rectangle(img,left,right,(0,255,0),thickness=2)
cv2.imwrite(r"C:\Users\sachu\Desktop\My_Documents\Research_Project\Computer_Graphics\CAR_TEST\test_image\Result\res.png",img)
cv2.imshow('Image',img)
cv2.waitKey(0) 
cv2.destroyAllWindows() 




